In [1]:
from bqplot import pyplot as plt
from bqplot import Lines, LinearScale, ColorScale, Tooltip, Scatter
import ipywidgets as widgets
import numpy as np
import pandas as pd
from ipywidgets import Output
from IPython.display import display, clear_output

In [2]:
!wget -nc https://datos.comunidad.madrid/catalogo/dataset/b3d55e40-8263-4c0b-827d-2bb23b5e7bab/resource/43708c23-2b77-48fd-9986-fa97691a2d59/download/covid19_tia_zonas_basicas_salud_s.csv 2> /dev/null

In [3]:
df = pd.read_csv("covid19_tia_zonas_basicas_salud_s.csv", sep=";", encoding='latin-1')
df["date"] = pd.to_datetime(df.fecha_informe)
df.tasa_incidencia_acumulada_ultimos_14dias = pd.to_numeric(df.tasa_incidencia_acumulada_ultimos_14dias.str.replace(",", "."))

In [4]:
f = plt.figure(title="Incidencia")
zonas = np.unique(df.zona_basica_salud)

#zones = np.unique(df.zona_basica_salud).tolist()
zones = ["Peñagrande", "Daroca"]
fdf = df[df.zona_basica_salud.isin(zones)]
data = [(zbs, partdf) for zbs, partdf in fdf.groupby("zona_basica_salud")]
ZBSs=[e[0] for e in data]
dfs = [e[1] for e in data]
xs = [d.date for d in dfs]
ys = [d.tasa_incidencia_acumulada_ultimos_14dias for d in dfs]
names = [d.zona_basica_salud for d in dfs]
x, y, names = np.stack(xs).reshape(-1), np.stack(ys).reshape(-1), np.stack(names).reshape(-1).tolist()



p1 = plt.plot(xs, ys, labels=ZBSs)
plt.legend()
colors = [[p1.colors[i % len(p1.colors)] for _ in range(len(d))] for i, d in enumerate(xs)] 
colors = np.stack(colors).reshape(-1).tolist()

p2 = plt.scatter(x=x,y=y, names=names, labels=names, display_names=False, colors=colors)
p2.tooltip = Tooltip(fields=['name','x', 'y'], formats = ['',"%d-%b-%y",''], labels = ['ZBS','Fecha', 'Incidencia'], show_labels = True )
p2.enable_hover = True
w = widgets.SelectMultiple(
    options=np.unique(df.zona_basica_salud).tolist(),
    value=np.unique(df.zona_basica_salud).tolist(),
    #rows=10,
    disabled=False
)

def on_change(args):
    global zones
    if args["new"] == True:
        zones = zones + [args["owner"].description]
    else:
        zones.remove(args["owner"].description)
    print(zones)

    fdf = df[df.zona_basica_salud.isin(zones)]
    data = [(zbs, partdf) for zbs, partdf in fdf.groupby("zona_basica_salud")]
    ZBSs=[e[0] for e in data]
    dfs = [e[1] for e in data]
    xs = [d.date for d in dfs]
    ys = [d.tasa_incidencia_acumulada_ultimos_14dias for d in dfs]
    names = [d.zona_basica_salud for d in dfs]
    x, y, names = np.stack(xs).reshape(-1), np.stack(ys).reshape(-1), np.stack(names).reshape(-1).tolist()
    
    p1.x = xs
    p1.y = ys
    p1.labels = ZBSs
    colors = [[p1.colors[i % len(p.colors)] for _ in range(len(d))] for i, d in enumerate(xs)] 
    colors = np.stack(colors).reshape(-1).tolist()
    
    p2.x = x
    p2.y = y
    p2.names = names
    p2.labels = names
    p2.colors = colors
    
ws = [widgets.Checkbox(value=zone in zones, description=zone) for zone in zonas]
[w.observe(on_change, "value") for w in ws]
widgets.VBox([f, *ws])
